Build and train a deep network for the fashion MNIST classification for the best accuracy. The goal is to get the best accuracy on the test data set.

We will use the same dataset from keras. 

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [1]:
import sys
import tensorflow as tf
assert tf.__version__ >= "2.0"
import numpy as np
import os
import matplotlib.pyplot as plt

from tensorflow import keras

from matplotlib import pyplot
from keras.datasets import fashion_mnist

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.3461 - accuracy: 0.6209 - val_loss: 0.9255 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.8197 - accuracy: 0.7355 - val_loss: 0.7305 - val_accuracy: 0.7628
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6966 - accuracy: 0.7694 - val_loss: 0.6565 - val_accuracy: 0.7878
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6331 - accuracy: 0.7909 - val_loss: 0.6004 - val_accuracy: 0.8046
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5917 - accuracy: 0.8057 - val_loss: 0.5656 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5618 - accuracy: 0.8135 - val_loss: 0.5406 - val_accuracy: 0.8240
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5390 - accuracy: 0.8206 - val_loss: 0.5196 - val_accuracy:

In [7]:
history.history['val_accuracy'][-1]

0.8398000001907349

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8293 - accuracy: 0.7221 - val_loss: 0.5539 - val_accuracy: 0.8160
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5703 - accuracy: 0.8035 - val_loss: 0.4792 - val_accuracy: 0.8378
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5161 - accuracy: 0.8212 - val_loss: 0.4424 - val_accuracy: 0.8490
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4789 - accuracy: 0.8315 - val_loss: 0.4212 - val_accuracy: 0.8566
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4548 - accuracy: 0.8404 - val_loss: 0.4050 - val_accuracy: 0.8610
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4387 - accuracy: 0.8445 - val_loss: 0.3931 - val_accuracy: 0.8638
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4254 - accuracy: 0.8504 - val_loss: 0.3829 - val_accuracy:

In [11]:
history.history['val_accuracy'][-1]

0.866599977016449

In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 7s 4ms/step - loss: 0.5905 - accuracy: 0.8074 - val_loss: 0.3892 - val_accuracy: 0.8655
Epoch 2/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3533 - accuracy: 0.8780 - val_loss: 0.3319 - val_accuracy: 0.8864
Epoch 3/20
1375/1375 [==============================] - 6s 5ms/step - loss: 0.3166 - accuracy: 0.8895 - val_loss: 0.3072 - val_accuracy: 0.8951
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2962 - accuracy: 0.8977 - val_loss: 0.2894 - val_accuracy: 0.9026
Epoch 5/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2824 - accuracy: 0.9019 - val_loss: 0.2818 - val_accuracy: 0.9061
Epoch 6/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2723 - accuracy: 0.9065 - val_loss: 0.2748 - val_accuracy: 0.9036
Epoch 7/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2644 - accuracy: 0.9096 - val_loss: 0.2658 - val_accuracy:

In [ ]:
model_A.save("my_model_A.h5")

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 45ms/step - loss: 0.5100 - accuracy: 0.7800 - val_loss: 0.4588 - val_accuracy: 0.8134
Epoch 2/20
7/7 [==============================] - 0s 18ms/step - loss: 0.3930 - accuracy: 0.8600 - val_loss: 0.3766 - val_accuracy: 0.8803
Epoch 3/20
7/7 [==============================] - 0s 15ms/step - loss: 0.3175 - accuracy: 0.9050 - val_loss: 0.3149 - val_accuracy: 0.9178
Epoch 4/20
7/7 [==============================] - 0s 19ms/step - loss: 0.2616 - accuracy: 0.9500 - val_loss: 0.2724 - val_accuracy: 0.9351
Epoch 5/20
7/7 [==============================] - 0s 18ms/step - loss: 0.2221 - accuracy: 0.9700 - val_loss: 0.2427 - val_accuracy: 0.9442
Epoch 6/20
7/7 [==============================] - 0s 17ms/step - loss: 0.1953 - accuracy: 0.9800 - val_loss: 0.2186 - val_accuracy: 0.9523
Epoch 7/20
7/7 [==============================] - 0s 20ms/step - loss: 0.1721 - accuracy: 0.9800 - val_loss: 0.2006 - val_accuracy: 0.9584
Epoch 8/20
7/7 [===========

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 43ms/step - loss: 1.0516 - accuracy: 0.2750 - val_loss: 0.9521 - val_accuracy: 0.3458
Epoch 2/4
7/7 [==============================] - 0s 17ms/step - loss: 0.9510 - accuracy: 0.3550 - val_loss: 0.8633 - val_accuracy: 0.4138
Epoch 3/4
7/7 [==============================] - 0s 16ms/step - loss: 0.8618 - accuracy: 0.4250 - val_loss: 0.7837 - val_accuracy: 0.4767
Epoch 4/4
7/7 [==============================] - 0s 14ms/step - loss: 0.7819 - accuracy: 0.5300 - val_loss: 0.7140 - val_accuracy: 0.5538
Epoch 1/16
7/7 [==============================] - 1s 43ms/step - loss: 0.5642 - accuracy: 0.7550 - val_loss: 0.3994 - val_accuracy: 0.8509
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.3407 - accuracy: 0.8850 - val_loss: 0.2825 - val_accuracy: 0.9290
Epoch 3/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2434 - accuracy: 0.9400 - val_loss: 0.2180 - val_accuracy: 0.9584
Epoch 4/16
7/7 [===============

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0669 - accuracy: 0.9950


[0.06687863171100616, 0.9950000047683716]

Accuracy : 0.9950

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=10,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=26,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/10
7/7 [==============================] - 0s 23ms/step - loss: 0.0546 - accuracy: 0.9900 - val_loss: 0.0670 - val_accuracy: 0.9888
Epoch 2/10
7/7 [==============================] - 0s 34ms/step - loss: 0.0519 - accuracy: 0.9900 - val_loss: 0.0645 - val_accuracy: 0.9888
Epoch 3/10
7/7 [==============================] - 0s 16ms/step - loss: 0.0495 - accuracy: 0.9900 - val_loss: 0.0623 - val_accuracy: 0.9888
Epoch 4/10
7/7 [==============================] - 0s 31ms/step - loss: 0.0474 - accuracy: 0.9900 - val_loss: 0.0604 - val_accuracy: 0.9888
Epoch 5/10
7/7 [==============================] - 0s 18ms/step - loss: 0.0455 - accuracy: 0.9900 - val_loss: 0.0585 - val_accuracy: 0.9888
Epoch 6/10
7/7 [==============================] - 0s 17ms/step - loss: 0.0438 - accuracy: 0.9900 - val_loss: 0.0567 - val_accuracy: 0.9888
Epoch 7/10
7/7 [==============================] - 0s 32ms/step - loss: 0.0421 - accuracy: 0.9950 - val_loss: 0.0551 - val_accuracy: 0.9888
Epoch 8/10
7/7 [===========

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.9975


[0.028272384777665138, 0.9975000023841858]

Accuracy : .9975

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=50,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=70,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/50
7/7 [==============================] - 0s 23ms/step - loss: 0.0195 - accuracy: 1.0000 - val_loss: 0.0343 - val_accuracy: 0.9899
Epoch 2/50
7/7 [==============================] - 0s 20ms/step - loss: 0.0191 - accuracy: 1.0000 - val_loss: 0.0340 - val_accuracy: 0.9899
Epoch 3/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0188 - accuracy: 1.0000 - val_loss: 0.0337 - val_accuracy: 0.9899
Epoch 4/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0185 - accuracy: 1.0000 - val_loss: 0.0334 - val_accuracy: 0.9899
Epoch 5/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0182 - accuracy: 1.0000 - val_loss: 0.0331 - val_accuracy: 0.9899
Epoch 6/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 0.0329 - val_accuracy: 0.9888
Epoch 7/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 0.0326 - val_accuracy: 0.9888
Epoch 8/50
7/7 [===========

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.9980


[0.013400241732597351, 0.9980000257492065]

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=100,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=120,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/100
7/7 [==============================] - 0s 24ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9899
Epoch 2/100
7/7 [==============================] - 0s 18ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9899
Epoch 3/100
7/7 [==============================] - 0s 19ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9899
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9899
Epoch 5/100
7/7 [==============================] - 0s 18ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0218 - val_accuracy: 0.9899
Epoch 6/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0218 - val_accuracy: 0.9899
Epoch 7/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9899
Epoch 8/100
7/7 [===

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0090 - accuracy: 0.9985


[0.00898818764835596, 0.9984999895095825]

Accuracy : 0.9985

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=20,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adagrad(learning_rate=1e-3), metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))


Epoch 1/20
7/7 [==============================] - 0s 25ms/step - loss: 4.5727e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 2/20
7/7 [==============================] - 0s 19ms/step - loss: 4.5614e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 3/20
7/7 [==============================] - 0s 33ms/step - loss: 4.5515e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 4/20
7/7 [==============================] - 0s 19ms/step - loss: 4.5410e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 4.5287e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 6/20
7/7 [==============================] - 0s 21ms/step - loss: 4.5192e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 7/20
7/7 [==============================] - 0s 32ms/step - loss: 4.5088e-04 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9929

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0057 - accuracy: 0.9985


[0.005740430671721697, 0.9984999895095825]

Accuracy : 0.9985